In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets loralib sentencepiece
!pip install -q gradio
!pip install -q ttsmms

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# For TTS

!curl https://dl.fbaipublicfiles.com/mms/tts/ben.tar.gz --output ben.tar.gz #update lang
!mkdir -p data && tar -xzf ben.tar.gz -C data/ #update langcode

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  128M  100  128M    0     0   127M      0  0:00:01  0:00:01 --:--:--  127M


In [3]:
from ttsmms import TTS

tts=TTS("data/ben") #update lang code

wav=tts.synthesis("আমি অলিভ, ওড়িয়া জেনারেল এআই গবেষকদের দ্বারা প্রশিক্ষিত একটি ভাষা মডেল")


In [4]:
from IPython.display import Audio
Audio(wav["x"], rate=wav["sampling_rate"])

In [5]:
import tempfile
from IPython.display import Audio


def tts_system(text: str):

    print(text)
    wavs = tts.synthesis(text)
    # return output
    af = Audio(wavs["x"], rate=wavs["sampling_rate"])
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as fp:
        fp.write(af.data)
        return fp.name

In [6]:
# import torch
from peft import PeftModel
# from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig


import torch
# from peft import PeftModel
import transformers
import gradio as gr

assert (
    "LlamaTokenizer" in transformers._import_structure["models.llama"]
), "LLaMA is now in HuggingFace's main branch.\nPlease reinstall it: pip uninstall transformers && pip install git+https://github.com/huggingface/transformers.git"
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, BitsAndBytesConfig

In [ ]:
model_id = "OdiaGenAI/odiagenAI-bengali-base-model-v1"
device = "cuda:0"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


tokenizer = LlamaTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")   # 4 bit model

def generate_prompt(instruction, input=None):
    if input:
        return f"""### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"""
    else:
        return f"""### Instruction:\n{instruction}\n\n### Response:\n"""


model.eval()
if torch.__version__ >= "2":
    model = torch.compile(model)


def evaluate(
    instruction,
    input=None,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=128,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    print(prompt)
    inputs = tokenizer(prompt, return_tensors="pt")
    print(inputs)
    input_ids = inputs["input_ids"].to(device)
    print(input_ids)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
        # print(generation_output)
    s = generation_output.sequences[0]
    print(s)
    output = tokenizer.decode(s, skip_special_tokens=True)
    print(output)

    text = output.split("### Response:")[1].strip()
    audio_wav = tts_system(text)
    return text, audio_wav
    # return output.split("### Response:")[1].strip()

input_text = gr.components.Textbox(
            lines=2, label="Instruction", placeholder="Tell me about alpacas."
        )
# tts_button = gr.Button("Change to voice", elem_id="send-btn", visible=True)

output_audio = gr.outputs.Audio(label="Output", type="filepath")

g = gr.Interface(
    fn=evaluate,
    inputs=[
        input_text,
        gr.components.Textbox(lines=2, label="Input", placeholder="none"),
        gr.components.Slider(minimum=0, maximum=1, value=0.1, label="Temperature"),
        gr.components.Slider(minimum=0, maximum=1, value=0.75, label="Top p"),
        gr.components.Slider(minimum=0, maximum=100, step=1, value=40, label="Top k"),
        gr.components.Slider(minimum=1, maximum=4, step=1, value=4, label="Beams"),
        gr.components.Slider(
            minimum=1, maximum=512, step=1, value=128, label="Max tokens"
        ),
    ],
    outputs=[
        gr.inputs.Textbox(
            lines=5,
            label="Output",
        ),
        output_audio
    ],
    title="🌊🐢 OdiaGenAI-4-bit-Bengali-LLM",
    description="OdiaGenAI-4-bit-Bengali-LLM is a 7B-parameter LLaMA model finetuned to follow Odia instructions. It is trained on the [Stanford Alpaca](https://github.com/tatsu-lab/stanford_alpaca) Odia translated dataset and makes use of the Huggingface LLaMA implementation. For more information, please visit [the project's website](https://www.odiagenai.org/).",
)



g.queue(concurrency_count=2)
g.launch(debug=True)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/34 [00:00<?, ?it/s]

<ipython-input-7-77ff3ba82ccc>:75: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  output_audio = gr.outputs.Audio(label="Output", type="filepath")
<ipython-input-7-77ff3ba82ccc>:91: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Textbox(
<ipython-input-7-77ff3ba82ccc>:91: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(
<ipython-input-7-77ff3ba82ccc>:91: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://fc55e3e8187de0c57a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### Instruction:
Tell me spmething about bangladesh

### Response:

{'input_ids': tensor([[    0,   835,  2799,  4080, 29901,    13, 29911,   514,   592,   805,
         29885,  1408,  1048,   289,   574, 29880, 21754,    13,    13,  2277,
         29937, 13291, 29901,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[    0,   835,  2799,  4080, 29901,    13, 29911,   514,   592,   805,
         29885,  1408,  1048,   289,   574, 29880, 21754,    13,    13,  2277,
         29937, 13291, 29901,    13]], device='cuda:0')


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.75` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configurati

tensor([    0,   835,  2799,  4080, 29901,    13, 29911,   514,   592,   805,
        29885,  1408,  1048,   289,   574, 29880, 21754,    13,    13,  2277,
        29937, 13291, 29901,    13, 30962, 30445,   227,   169,   133, 31055,
        30445, 31204, 30932, 31686, 29871,   227,   169,   146, 30995,   227,
          169,   162, 30786, 29871, 31204, 30995, 30655,   227,   169,   186,
        30786, 30792, 29871,   227,   169,   137, 31006, 30932, 30653, 30786,
        30995, 30445, 30792, 29871, 31204, 30932, 31686, 29892, 29871, 31218,
        30445, 30653, 29871,   227,   169,   140, 31150, 30655, 31150, 30653,
        30932, 29871,   227,   169,   176, 30445, 30653, 31150, 29871,   227,
          169,   146, 30962,   227,   169,   133, 29871, 31006, 30445, 30653,
        30655, 30995, 30786, 30792, 29871, 31218, 31154, 30995, 30655, 31150,
        30653, 30445,   227,   169,   186, 30655,   227,   169,   162, 30655,
        30653, 30932, 30653, 29871, 31006,   227,   169,   170, 

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_ut

tensor([    0,   835,  2799,  4080, 29901,    13,    13,    13,  2277, 29937,
        10567, 29901,    13, 29887,   573,   592,   278,  7379,  1036,  2181,
         8522,  8158, 29889,    13,    13,  2277, 29937, 13291, 29901,    13,
          227], device='cuda:0')
### Instruction:


### Input:
give me the todays cricket score.

### Response:
�
�
### Instruction:


### Input:
tell me about cancer

### Response:

{'input_ids': tensor([[    0,   835,  2799,  4080, 29901,    13,    13,    13,  2277, 29937,
         10567, 29901,    13, 29873,   514,   592,  1048, 23900,    13,    13,
          2277, 29937, 13291, 29901,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}
tensor([[    0,   835,  2799,  4080, 29901,    13,    13,    13,  2277, 29937,
         10567, 29901,    13, 29873,   514,   592,  1048, 23900,    13,    13,
          2277, 29937, 13291, 29901,    13]], device='cuda:0')
tensor([    0,   835,  2799, 